# Monte Carlo Simulation

In [1]:
from classes.MyDataLogger import MyDataLogger, MySubDataLogger
from modules.helper_functions_tsp import (find_problem_size,
                                          cost_fn_fact, 
                                          find_distances_array,
                                          convert_integer_to_binary_list,
                                          )
from itertools import count
import random


In [2]:
from modules.config import (GRAY,
                            DECODING_FORMULATION
                            )

Instantiate datalogger

In [3]:
datalogger = MyDataLogger()
sdl_list = []



In [4]:
#LOCATIONS = [4, 5, 6, 7, 8, 9, 10, 11, 12, 15, 17, 26, 42, 48]
LOCATIONS = [12, ]

ITERATIONS = 250
QUANTUM = True

In [5]:
if not isinstance(QUANTUM, bool):
    raise TypeError("QUANTUM must be a Boolean (True or False)")

In [6]:
#entered manually from show_results.ipynb
#maximum cache size for Circuits 1, 2, 3, 4, 6 by location
if QUANTUM:
   cache_size = {4 : 8_016,
               5 : 30_336,
               6 : 170_262,
               7 : 407_199,
               8 : 807_210,
               9 : 953_795,
               10 : 1_014_396,
               11 : 1_012_825,
               12 : 1_025_235
                  }
else:
   cache_size = {4 : 1_028_096,
               5 : 1_542_144,
               6 : 2_313_216,
               7 : 3_084_288,
               8 : 3_855_360,
               9 : 4_626_432,
               10 : 5_654_528,
               11 : 6_682_624,
               12 : 7_710_720,
               15 : 10_795_008,
               17 : 12_851_200,
               26 : 24_417_280,
               42 : 47_292_416, 
               48 : 56_545_280,
                  }

Main loop

In [7]:
print(f'Running Monte Carlo simulations for {DECODING_FORMULATION} formulation with Gray = {GRAY}.')
for iterations in range(ITERATIONS):
    print(f'Iteration {iterations} of {ITERATIONS}.')
    for i in range(len(LOCATIONS)):
        sdl = MySubDataLogger(runid = datalogger.runid)
        sdl.quantum = QUANTUM
        sdl.locations = LOCATIONS[i]
        sdl.formulation = DECODING_FORMULATION
        sdl.gray = GRAY
        sdl.hot_start = False
        sdl.monte_carlo = True
        qubits = find_problem_size(sdl.locations, sdl.formulation)
        sdl.qubits = qubits
        print(f'There are {qubits} qubits needed for {sdl.locations} locations in the {sdl.formulation} formulation.')
        distance_array, best_dist = find_distances_array(sdl.locations, 
                                                        print_comments=False,
                                                        )
        sdl.best_dist = best_dist 
        cost_fn = cost_fn_fact(sdl.locations,
                                distance_array, 
                                sdl.gray, 
                                method=sdl.formulation,
                            )
        sdl.cache_max_size = cache_size[sdl.locations]
        lowest_to_date = 999_999 #too high
        highest = 2**sdl.qubits-1
        for j in range(sdl.cache_max_size):
            integer = random.randint(1, highest)
            binary_string_list = convert_integer_to_binary_list(integer, 
                                                                sdl.qubits, 
                                                                gray = sdl.gray)
            cost = cost_fn(binary_string_list)
            if cost < lowest_to_date:
                lowest_to_date = cost
                iterations_found = j
        
        print(f' The lowest cost for {sdl.locations} locations is {lowest_to_date} which was found after {iterations_found} iterations out of {sdl.cache_max_size}.')
        sdl.best_dist_found = lowest_to_date
        sdl.update_cache_statistics(cost_fn)
        print(f'Cache hits: {sdl.cache_hits}, cache misses: {sdl.cache_misses}, total cache calls: {sdl.cache_hits + sdl.cache_misses}.')
        sdl.save_results_to_csv()

Running Monte Carlo simulations for original formulation with Gray = False.
Iteration 0 of 250.
SubDataLogger instantiated.  Run ID = 20250801-17-13-37 - 17-13-37
There are 29 qubits needed for 12 locations in the original formulation.
Reading distance data
 The lowest cost for 12 locations is 340.6 which was found after 574958 iterations out of 1025235.
Cache hits: 1022, cache misses: 1024213, total cache calls: 1025235.
Saving data to results\results.csv
Iteration 1 of 250.
SubDataLogger instantiated.  Run ID = 20250801-17-13-37 - 17-14-28
There are 29 qubits needed for 12 locations in the original formulation.
Reading distance data
 The lowest cost for 12 locations is 297.2 which was found after 446936 iterations out of 1025235.
Cache hits: 975, cache misses: 1024260, total cache calls: 1025235.
Saving data to results\results.csv
Iteration 2 of 250.
SubDataLogger instantiated.  Run ID = 20250801-17-13-37 - 17-15-26
There are 29 qubits needed for 12 locations in the original formulat